In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN # 기본 RNN. 실제로는 사용하지 않는다. LSTM, GRU 등을 사용

model = Sequential([
    SimpleRNN(3, input_shape=(2,10)) # 은닉 유닛 : 3, timestep : 2, embedding dim : 10
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# 네이버 영화 리뷰 분류 모델 만들기

In [19]:
# 필요 패키지
import pandas as pd
import numpy as np
import urllib.request

import matplotlib.pyplot as plt

# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fdb90cde200>)

In [20]:
df_train = pd.read_table('ratings_train.txt')
df_test  = pd.read_table('ratings_test.txt')

df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


# 전처리 수행

In [22]:
# 중복 데이터 제거
df_train['document'].nunique()

146182

### 약 4천개의 데이터가 중복된 리뷰

In [23]:
df_train = df_train.drop_duplicates(subset=['document']) # drop_duplicates : 중복된 데이터 제거
df_train.nunique()

id          146183
document    146182
label            2
dtype: int64

In [24]:
# NaN값 제거
df_train = df_train.dropna(how='any') # NaN이 어디에든 들어 있으면 제거
df_train.isnull().values.sum() #  NaN 값의 개수를 세었을 때 0이면 NaN이 없음!

0

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146182 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146182 non-null  int64 
 1   document  146182 non-null  object
 2   label     146182 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [26]:
# 정규식을 이용해 한글만 추출하기
# str을 안쓰면 document컬럼의 속성을 문자열로 변경하고 re모듈을 사용해야한다.
df_train['document']= df_train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","").str.strip()
df_train.head()

<ipython-input-26-9f7d579f1d8d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train['document']= df_train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","").str.strip()


,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소이야기구먼솔직히재미는없다평점조정,0
4,6483659,사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...,1


In [27]:
# 한글을 제외하고 나머지 문자열들은 전부 비어있는 문자열 ''로 치환함.
#  영어만 있거나, 숫자만 있거나, 한글이 포함되지 않은 리뷰가 남아 있을 수 있음!
df_train.loc[df_train['document'] == '', 'document'].value_counts()

    789
Name: document, dtype: int64

In [28]:
# 비어있는 문자열을 nan으로 치환해서 drop
df_train['document'].replace("",np.nan,inplace=True)
df_train = df_train.dropna(how='any')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145393 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        145393 non-null  int64 
 1   document  145393 non-null  object
 2   label     145393 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.4+ MB


# 토큰화 수행

In [29]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.5 MB/s eta 0:00:00


In [33]:
from tqdm import tqdm_notebook
from konlpy.tag import Okt

okt=Okt()

X_train = []

# 코퍼스 생성을 위한 리스트
for sentence in tqdm_notebook(df_train['document']):
  # 형태소 분리
  temp_X = okt.morphs(sentence, stem=True, norm=True)
  X_train.append(temp_X)

<ipython-input-33-57bd22957dc4>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(df_train['document']):


  0%|          | 0/145393 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
import pickle

with open("X_train.pkl", "wb") as f:
  pickle.dump(X_train, f)

In [ ]:
# 생성된 코퍼스 확인
X_train[:3]

## 토크나이저 생성

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

### 단어 빈도수 확인
- 빈도수가 적은 단어는 사용 X
- 오타, 신조어 등 일반적으로 사용하지 않는 단어는 제거

In [ ]:
# threshold 만큼 등장하지 않은 단어들의 비율 확인
def print_freq(threshold):
  total_cnt = len(tokenizer.word_index) # 단어 개수
  rare_cnt  = 0 # 희귀 단어 개수. 등장 빈도가 threshold보다 작은 단어의 개수를 카운트

  total_freq = 0 # 훈련 데이터 전체 단어 빈도수 총 합. ex) 영화 : 3번, 재미 : 5번 -> total_freq = 8
  rare_freq  = 0 # 사용된 희귀 단어 빈도의 총 합

  # word_counts : (단어, 단어 빈도)
  for word, frequency in tokenizer.word_counts.items():

    total_freq = total_freq + frequency # 모든 단어의 빈도수 총 합 구하기

    # 희귀 단어 개수 및 등장 빈도 계산
    if frequency < threshold:
      rare_cnt += 1 # 희귀 단어 개수 +1
      
      rare_freq += frequency # 등장한 희귀 단어의 빈도수 총 합

  print("전체 단어 집합의 크기 : {}".format(total_cnt))
  print("등장 빈도가 {} 미만인 희귀 단어의 수 : {}".format(threshold, rare_cnt))
  print("단어 집합에서 희귀 단어의 비율 : {:.3f}%".format((rare_cnt / total_cnt) * 100))
  print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : {:.3f}%".format((rare_freq / total_freq) * 100))
  print("="*50)

  return total_cnt, rare_cnt

In [ ]:
print_freq(2)
print_freq(3)
print_freq(4)
print_freq(5)

### 새로운 토크나이져 만들기
 - 희귀단어 제거, oov, padding 고려해서 토크나이저 만들기

In [ ]:
# vocab_size : 사용할 단어의 개수
# 전체 단어의 개수 - 희귀 단어의 개수
total_cnt, rare_cnt = print_freq(3)
vocab_size = total_cnt - rare_cnt + 2 # <oov>, <pad> 고려해서 +2
print(vocab_size)

In [ ]:
# 희귀 단어 제거 후 토크나이져 만들기
tokenizer = Tokenizer(vocab_size, oov_token='<oov>')
tokenizer.fit_on_texts(X_train)

In [ ]:
# 정수 인코딩
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_train_encoded[:3]

## 비어 있는 배열 확인
- 빈도수 제한을 통해서 토크나이져를 생성하면 희귀 단어로만 이루어져 있던 코퍼스는 사용 X
- 비어있는 배열로 존재
  - [희귀, 희귀, 희귀] : [ ]

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train_encoded) if len(sentence) < 1]

In [ ]:
X_train_encoded = np.delete(X_train_encoded, drop_train, axis=0)
X_train_encoded[:3]

In [ ]:
y_train = np.array(df_train['label'])
y_train = np.delete(y_train, drop_train, axis=0)
y_train

## 패딩 설정하기

In [ ]:
print("리뷰의 최대 길이 : {}".format(max(len(l) for l in X_train_encoded)))
print("리뷰의 평균 길이 : {}".format(sum(map(len, X_train_encoded)) / len(X_train_encoded)))

In [ ]:
# 시각화
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel("Length Of Samples")
plt.ylabel("Number Of Samples")
plt.show()

In [ ]:
def below_threshold_len(max_len):
  cnt = 0

  # max_len을 넘지 않는 문장의 개수 확인
  for s in X_train_encoded:
    if len(s) <= max_len:
      cnt += 1

  print("전체 샘플 중 길이가 {} 이하인 샘플의 비율 : {:.3f}%".format(max_len, (cnt / len(X_train_encoded) * 100)))

In [ ]:
below_threshold_len(20)
below_threshold_len(30)
below_threshold_len(40)
below_threshold_len(50)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_padded = pad_sequences(X_train_encoded, maxlen=30)

In [ ]:
X_train_encoded[:3]

In [ ]:
X_train_padded[:3]

# LSTM으로 리뷰 분류기 만들기

In [31]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [42]:
model = Sequential([
    Embedding(19260,128), # (단어의 개수,임베딩차원)
    LSTM(128), # 의도 학습
    Dense(1, activation='sigmoid') # 의도 추론, 유닛한개, 이진분류(긍정,부정)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         2465280   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,596,993
Trainable params: 2,596,993
Non-trainable params: 0
_________________________________________________________________


In [43]:
import pickle
with open("./X_train_padded.pkl", 'rb') as f:
  X_train_padded = pickle.load(f)

with open("./t_train.pkl", 'rb') as f:
  t_train = pickle.load(f)

with open("./X_train.pkl", 'rb') as f:
  X_train = pickle.load(f)

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 19260

# 희귀 단어 제거 후 토크나이져 만들기
tokenizer = Tokenizer(vocab_size, oov_token='<oov>')
tokenizer.fit_on_texts(X_train)

In [45]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # 시그모이드를 썼기 때문에
    metrics=['acc']
)

In [46]:
model.fit(
    X_train_padded,
    t_train,
    batch_size=128,
    validation_split=0.2,
    epochs=10
)

Epoch 1/10
909/909 [==============================] - 63s 56ms/step - loss: 0.3922 - acc: 0.8222 - val_loss: 0.3529 - val_acc: 0.8461
Epoch 2/10
909/909 [==============================] - 19s 20ms/step - loss: 0.3038 - acc: 0.8697 - val_loss: 0.3405 - val_acc: 0.8532
Epoch 3/10
909/909 [==============================] - 11s 13ms/step - loss: 0.2585 - acc: 0.8911 - val_loss: 0.3463 - val_acc: 0.8530
Epoch 4/10
909/909 [==============================] - 9s 10ms/step - loss: 0.2188 - acc: 0.9097 - val_loss: 0.3671 - val_acc: 0.8481
Epoch 5/10
909/909 [==============================] - 10s 11ms/step - loss: 0.1833 - acc: 0.9262 - val_loss: 0.4470 - val_acc: 0.8451
Epoch 6/10
909/909 [==============================] - 9s 10ms/step - loss: 0.1550 - acc: 0.9378 - val_loss: 0.4615 - val_acc: 0.8414
Epoch 7/10
909/909 [==============================] - 8s 9ms/step - loss: 0.1284 - acc: 0.9495 - val_loss: 0.5360 - val_acc: 0.8416
Epoch 8/10
909/909 [==============================] - 8s 9ms/step 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 일반 자연어(새로운 문장)가 들어 왔을 때 모델을 이용한 처리
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  
  # 정수 인코딩
  encoded = tokenizer.texts_to_sequences([new_sentence])
  
  # 패딩
  pad_new = pad_sequences(encoded, maxlen=30)

  # 예측
  score = float(model.predict(pad_new))

  # 시그모이드를 사용했기 때문에 결과값이 0.5 넘어가면 긍정 리뷰(1)
  if score > 0.5 :
    print("{:.2f}% 확률로 긍정 리뷰 입니다.".format(score * 100))
  else:
    # 0.5 미만이면 부정 리뷰 (0)
    print("{:.2f}% 확률로 부정 리뷰 입니다.".format(( 1 - score ) * 100))